In [31]:
import requests
import bs4
from bs4 import BeautifulSoup

URLS = []
with open('top_10.txt', mode='r') as top10:
    for illness in top10:
        URLS.append(illness.strip())
        


In [32]:
URLS

['https://www.mayoclinic.org/diseases-conditions/atrial-fibrillation/symptoms-causes/syc-20350624',
 'https://www.mayoclinic.org/diseases-conditions/copd/symptoms-causes/syc-20353679',
 'https://www.mayoclinic.org/diseases-conditions/alzheimers-disease/symptoms-causes/syc-20350447',
 'https://www.mayoclinic.org/diseases-conditions/depression/symptoms-causes/syc-20356007',
 'https://www.mayoclinic.org/diseases-conditions/heart-failure/symptoms-causes/syc-20373142',
 'https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521',
 'https://www.mayoclinic.org/diseases-conditions/type-2-diabetes/symptoms-causes/syc-20351193',
 'https://www.mayoclinic.org/diseases-conditions/heart-disease/symptoms-causes/syc-20353118',
 'https://www.mayoclinic.org/diseases-conditions/arthritis/symptoms-causes/syc-20350772',
 'https://www.mayoclinic.org/diseases-conditions/high-blood-cholesterol/symptoms-causes/syc-20350800',
 'https://www.mayoclinic.org/diseases-conditi

In [44]:
top10_symptoms = []
for URL in URLS:
    top10_symptoms.append(get_symptoms(URL))

Got one!
Got one!


In [45]:
top10_symptoms

[{'illness': 'Atrial fibrillation',
  'symptoms': "Some people with atrial fibrillation (A-fib) don't notice any symptoms. Those who do have atrial fibrillation symptoms may have signs and symptoms such as: Sensations of a fast, fluttering or pounding heartbeat (palpitations) Chest pain Dizziness Fatigue Lightheadedness Reduced ability to exercise Shortness of breath Weakness Atrial fibrillation may be: Occasional (paroxysmal atrial fibrillation). A-fib symptoms come and go, usually lasting for a few minutes to hours. Sometimes symptoms occur for as long as a week and episodes can happen repeatedly. Symptoms might go away on their own. Some people with occasional A-fib need treatment. Persistent. With this type of atrial fibrillation, the heart rhythm doesn't go back to normal on its own. If a person has A-fib symptoms, cardioversion or treatment with medications may be used to restore and maintain a normal heart rhythm. Long-standing persistent. This type of atrial fibrillation is con

In [48]:
import json

with open('updated_illness.json', 'w') as data:
    json.dump(top10_symptoms, data)

In [43]:
def get_symptoms(URL):
    r = requests.get(URL)

    soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib
#     print(soup.prettify())

    name = soup.find('h1').get_text()
    
    content = soup.find('div', class_='content')
    content = content.find('h2').previous_element.previous_element.previous_element
    
    h2_list = content.find_all('h2')
    
    symp_index = -1
    for i in range(len(h2_list)):
        if h2_list[i].get_text() == 'Symptoms':
            symp_index = i
            break

    if symp_index == -1:
        return {}
    else:
        visually_hidden = content.find_all("span", {"class": "myc-visuallyhidden"})
        weird_error = content.find_all("p", {"class": "myc-error-header"})
        
        h2_index = symp_index
        element = h2_list[h2_index]

        parser_dict = {}
        parser_dict['illness'] = name
        section_str = ''

        while h2_index == symp_index:
            element = element.next_element
            if element == h2_list[h2_index + 1]:
                parser_dict[h2_list[h2_index].get_text().lower()] = section_str.replace("  ", " ").replace(" , ", ", ").replace(" . ", ". ")
                h2_index += 1
                section_str = ''

        #         if element.get_text() == '':
        #             empty_header = True

            elif isinstance(element, bs4.element.NavigableString):
                if not isinstance(element, bs4.element.Comment):
                    if (element != h2_list[h2_index].get_text()):
                        if not element.isspace():
                            section_str += element.strip() + ' '

            elif element in visually_hidden:
                element = element.next_element
                print('Got one!')

            elif element in weird_error:
                element = element.next_element
                print('Oh!')
                
        return parser_dict

In [4]:
content = soup.find('div', class_='content')
# content = soup.find('div', class_='contentbox')
# content = content.next_element.next_element.next_element



In [5]:
content = content.find('h2').previous_element.previous_element.previous_element

In [36]:
content

<div>
    

    
            
            <!--googleoff: all--><h2>Overview</h2><!--googleon: all--><div class="access-modal right p-3">


<!--googleoff: all-->
<button class="modal-trigger image-lightbox" id="et_ds4164aab_tg88233438" onclick="openDialog('dialogId6775719', this, 'label6775719', 'modal-lightbox'); return false;">
	<img alt="Coronavirus" aria-hidden="true" src="/-/media/kcms/gbs/patient-consumer/images/2020/02/12/16/17/coronovirus-8col.jpg"/>

    <span>
	    Coronavirus
	    <span class="myc-visuallyhidden">Open pop-up dialog box</span>
    </span>

	
</button>

<!--googleon: all-->
<div class="dialog-backdrop">
	<div aria-labelledby="label6775719" aria-modal="true" class="hidden" data-url-tracking="true" id="dialogId6775719" role="dialog">
		<div class="image-lightbox-container">
			<!--googleoff: all-->
			<div class="dialog_form_actions lightbox-close">
				<button class="modal-close" onclick="closeDialog(this, event)">
					
						<span class="caption myc-visuallyhi

In [7]:
h2_list = content.find_all('h2')
h2_list

[<h2>Overview</h2>,
 <h2>Symptoms</h2>,
 <h2 class="eh-title-main" id="formTitle">Get the latest health information from Mayo Clinic’s experts.
                         </h2>,
 <h2 data-msg="Your Form is Being Submitted" id="myTitle"></h2>,
 <h2>Causes</h2>,
 <h2>Risk factors</h2>,
 <h2>Complications</h2>,
 <h2>Prevention</h2>,
 <h2>Related information</h2>,
 <h2></h2>]

In [8]:
symp_index = -1
for i in range(len(h2_list)):
    if h2_list[i].get_text() == 'Symptoms':
        symp_index = i
        break
    
# If -1 error?

In [310]:
# Start of symptoms
type(h2_list[1].next_element.next_element.next_element)

bs4.element.NavigableString

In [9]:
visually_hidden = content.find_all("span", {"class": "myc-visuallyhidden"})
visually_hidden

[<span class="myc-visuallyhidden">Open pop-up dialog box</span>,
 <span class="caption myc-visuallyhidden">Close</span>,
 <span class="myc-visuallyhidden">Error</span>,
 <span class="myc-visuallyhidden">Error</span>]

In [11]:
weird_error = content.find_all("p", {"class": "myc-error-header"})
weird_error

[<p class="myc-error-header" id="errorBanner" role="alert">There is a problem with
                                 information submitted for this request. Review/update the
                                 information highlighted below and resubmit the form.</p>]

In [18]:
h2_index = symp_index
element = h2_list[h2_index]

parser_dict = {}
section_str = ''

# empty_header = False
while h2_index == symp_index:
    element = element.next_element
#     print(element)
#     print(type(element))
    if element == h2_list[h2_index + 1]:
        parser_dict[h2_list[h2_index].get_text()] = section_str.replace("  ", " ").replace(" , ", ", ").replace(" . ", ". ")
        h2_index += 1
        section_str = ''
        
#         if element.get_text() == '':
#             empty_header = True
            
            
    elif isinstance(element, bs4.element.NavigableString):
        if not isinstance(element, bs4.element.Comment):
            if (element != h2_list[h2_index].get_text()):
                if not element.isspace():
                    section_str += element.strip() + ' '
    
    elif element in visually_hidden:
        element = element.next_element
        print('Got one!')
        
    elif element in weird_error:
        element = element.next_element
        print('Oh!')


Oh!


In [19]:
h2_index

2

In [20]:
parser_dict

{'Symptoms': "Signs and symptoms of coronavirus disease 2019 (COVID-19) may appear 2 to 14 days after exposure. This time after exposure and before having symptoms is called the incubation period. You can still spread COVID-19 before you have symptoms (presymptomatic transmission). Common signs and symptoms can include: Fever Cough Tiredness Early symptoms of COVID-19 may include a loss of taste or smell. Other symptoms can include: Shortness of breath or difficulty breathing Muscle aches Chills Sore throat Runny nose Headache Chest pain Pink eye (conjunctivitis) Nausea Vomiting Diarrhea Rash This list isn't complete. Children have similar symptoms to adults and generally have mild illness. The severity of COVID-19 symptoms can range from very mild to severe. Some people may have only a few symptoms. Some people may have no symptoms at all, but can still spread it (asymptomatic transmission). Some people may experience worsened symptoms, such as worsened shortness of breath and pneumon

In [113]:
copium = soup.find('div', class_='contentbox')
copium = copium.next_element.next_element.next_element

In [110]:
copium.find_all('h2')

[<h2>Overview</h2>,
 <h2>Symptoms</h2>,
 <h2>Causes</h2>,
 <h2>Risk factors</h2>,
 <h2>Complications</h2>,
 <h2>Prevention</h2>,
 <h2></h2>]

In [143]:
listings = ''

for ul in copium.find_all('ul'):
    print(ul)
    print()
    try:
        if ul['class'] == 'listGroup':
            continue
    except KeyError: # element does not have a class
        li = ul.find_all("li")
        for l in li:
            listings += l.get_text() + ' '
        
print(listings)

<ul class="listGroup" id="ul_e6d09fbd-19fe-49ac-9b47-bd857c0d411b"><li class="acces-listitems"><a href="https://order.store.mayoclinic.com/books/gnweb43?utm_source=MC-DotOrg-PS&amp;utm_medium=Link&amp;utm_campaign=FamilyHealth-Book&amp;utm_content=FHB">Book: Mayo Clinic Family Health Book, 5th Edition</a></li><li class="acces-hide-listitems"><a href="https://order.store.mayoclinic.com/hl/hldiged?utm_source=MC-DotOrg-PS&amp;utm_medium=Link&amp;utm_campaign=HealthLetter-Digital&amp;utm_content=HLDE">Newsletter: Mayo Clinic Health Letter — Digital Edition</a></li></ul>

<ul>
    <li>Sensations of a fast, fluttering or pounding heartbeat (palpitations)</li>
    <li>Chest pain</li>
    <li>Dizziness</li>
    <li>Fatigue</li>
    <li>Lightheadedness</li>
    <li>Reduced ability to exercise</li>
    <li>Shortness of breath</li>
    <li>Weakness</li>
</ul>

<ul>
    <li><strong>Occasional (paroxysmal atrial fibrillation).</strong> <abbr title="Atrial fibrillation">A-fib</abbr> symptoms come an